# ASCOR database creation 

This notebook creates the SQL database for the ASCOR data.

# Necessary downloads

In [4]:
import sys
import os
import re
import pandas as pd 
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import pandas as pd
import os
import re
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
# Add the project root to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Now import with full module path
from utils.database_creation_utils import get_db_connection, get_engine

engine = get_engine()



# Creating Country Entity

THIS ADDS IT TO ASCOR BECAUSE OF THE ENV FILE MAKE THE ENGONE FUNCITON BE ALE TO SPECIFCY WHAT TO CONNECT TO 

In [ ]:

# Load the Excel file
df = pd.read_excel("../data/TPI_ASCOR_data_13012025/ASCOR_countries.xlsx")
df.columns = df.columns.str.strip()

# Drop and create the country table
create_country_sql = """
DROP TABLE IF EXISTS country;

CREATE TABLE country (
  country_name VARCHAR NOT NULL,
  country_id INT NOT NULL,
  iso VARCHAR,
  region VARCHAR,
  bank_lending_group VARCHAR,
  imf_category VARCHAR,
  un_party_type VARCHAR,
  PRIMARY KEY (country_name, country_id)
);
"""

with engine.connect() as conn:
    conn.execute(text(create_country_sql))
    conn.commit()

# Select and rename columns to match database schema
country_df = df[[
    'Name',
    'Id',
    'Country ISO code',
    'Region',
    'World Bank lending group',
    'International Monetary Fund fiscal monitor category',
    'Type of Party to the United Nations Framework Convention on Climate Change'
]].copy()

country_df.columns = [
    'country_name', 'country_id', 'iso', 'region',
    'bank_lending_group', 'imf_category', 'un_party_type'
]

# Insert into database
country_df.to_sql("country", engine, if_exists="append", index=False)

print("Country table created and populated successfully.")


✅ Country table created and populated successfully.
